# Regex in Pandas

**Inhalt:** Regular Expressions in Pandas anwenden

**Nötige Skills:** Regex in Python

**Lernziele:**
- Ein praktisches Beispiel kennenlernen, wo Regex nützlich sein kann

# Das Beispiel

Das Bundesamt für Statistik stellt oft Dateien in verknorkster Form zur Verfügung, zum Beispiel, wenn man Auswertungen nach Gemeinden, Bezirken und Kantonen über das interaktive Portal generieren lässt: https://www.pxweb.bfs.admin.ch/pxweb/de/

Das vorliegende Beispiel beinhaltet die Bevölkerungszahlen, gegliedert nach Zivilstand einerseits (ledig, verheiratet, etc) und nach räumlicher Struktur (Gemeinden, Bezirke, Kantone) andererseits.

Das File ist gespeichert unter `dataprojects/BFS/px-x-0102010000_103.xlsx`

Unser 1. Ziel ist: Wir wollen die Tabelle filtern
- nur die Gemeinden
- nur die Bezirke
- nur die Kantone

Unser 2. Ziel ist: Wir wollen bei den Gemeinden die jeweilige Nummer isolieren

## Vorbereitung

In [1]:
import pandas as pd

In [2]:
import re

In [3]:
%matplotlib inline

## Datei laden

In [4]:
path = 'dataprojects/BFS/px-x-0102010000_103.xlsx'

In [5]:
df = pd.read_excel(path)

## Explorieren

In [6]:
df.head(20)

,Einheit,Zivilstand,Anzahl
0,Schweiz,Ledig,3650651
1,Schweiz,Verheiratet,3583008
2,Schweiz,Verwitwet,407408
3,Schweiz,Geschieden,685622
4,- Zürich,Ledig,666873
5,- Zürich,Verheiratet,610396
6,- Zürich,Verwitwet,63173
7,- Zürich,Geschieden,125889
8,>> Bezirk Affoltern,Ledig,21785
9,>> Bezirk Affoltern,Verheiratet,23865


Welche Einheitstypen gibt es? Und welches Muster haben sie?
- Gemeinde ("...... 9999 Gemeindename")
- Bezirk (">> Bezirsname")
- Kantone ("- Kantonsname")
- Land ("Land")

## Pandas-Funktionen, die Regex brauchen können

Einige Befehle heissen leicht anders, funktionieren aber sehr ähnlich wie in der re.Library

- **`str.contains(r"regex")`**: das Pendant zu `re.search()` - ja/nein-Antwort

- **`str.extract(r"regex")`**: auch ähnlich wie `re.search()` - Suchergebnis als Antwort

- **`str.replace(r"regex", "str")`**: das Pendant zu `re.sub()` - ersetzt Match mit String

Wir wenden diese Funktionen jetzt an.

### Aber zuerst ...

**Quiz:** Zuerst brauchen wir die Regex-Ausdrücke, um die Einheiten zu erkennen.

Am besten mit Tests beginnen, ob die Regex an einer Einheit anschlägt - und jeweils auch testen, ob die Regex bei Einheiten, die wir *nicht* wollen, auch *nicht* anschlägt.

**Test für Kantone**

In [9]:
# Unsere Regex
regex_k = r"-\s\w*\b"
#oder simpler: r"^-"

In [10]:
# Schlägt die regex bei den Kantonen an?
# (Gut ist, wenn wir ein "Match"-Objekt erhalten)
re.search(regex_k, "- Aargau")

<re.Match object; span=(0, 8), match='- Aargau'>

In [11]:
# Schlägt die regex bei den Bezirken NICHT an?
# (Gut ist hier, wenn wir nichts zurückerhalten)
re.search(regex_k, ">> Wahlkreis Luzern-Stadt")

In [12]:
# Schlägt die regex bei den Gemeinden NICHT an?
re.search(regex_k, "......0001 Aeugst am Albis")

**Test für Bezirke**

In [13]:
# Unsere Regex
regex_b = r"^>>"

In [15]:
# Schlägt die regex bei den Bezirken an?
re.search(regex_b, "- Aargau")

In [16]:
# Schlägt die regex bei den Kantonen NICHT an?
re.search(regex_b, ">> Wahlkreis Luzern-Stadt")

<re.Match object; span=(0, 2), match='>>'>

In [17]:
# Schlägt die regex bei den Gemeinden NICHT an?
re.search(regex_b, "......0001 Aeugst am Albis")

**Test für Gemeinden**

In [18]:
# Unsere Regex
regex_g = r"\.{6}"

In [19]:
# Test für Gemeinde
re.search(regex_g, "......0001 Aeugst am Albis")

<re.Match object; span=(0, 6), match='......'>

In [20]:
# Schlägt die regex bei den Kantonen NICHT an?
re.search(regex_g, ">> Wahlkreis Luzern-Stadt")

In [21]:
# Schlägt die regex bei den Bezirken NICHT an?
re.search(regex_g, "- Aargau")

## 1. Daten filtern

Nachdem wir nun die passenden Regex-Ausdrücke gefunden haben, können wir unsere Daten filtern.

Ziel: Wir erstellen ein separates DF für Gemeinden, Bezirke, Kantone.

Hier können wir die Funktion `str.contains()` gut brauchen.

Wir testen damit mal, ob die Einträge in der Spalte "Einheit" eine Gemeinde sind:

In [22]:
df['Einheit'].str.contains(regex_g)

0       False
1       False
2       False
3       False
4       False
        ...  
9991     True
9992     True
9993     True
9994     True
9995     True
Name: Einheit, Length: 9996, dtype: bool

Basierend auf dieser True/False-Liste können wir nun die Tabelle filtern.

In [23]:
df[df['Einheit'].str.contains(regex_g)]

,Einheit,Zivilstand,Anzahl
12,......0001 Aeugst am Albis,Ledig,852
13,......0001 Aeugst am Albis,Verheiratet,908
14,......0001 Aeugst am Albis,Verwitwet,74
15,......0001 Aeugst am Albis,Geschieden,147
16,......0002 Affoltern am Albis,Ledig,5000
...,...,...,...
9991,......6809 Haute-Ajoie,Geschieden,69
9992,......6810 La Baroche,Ledig,490
9993,......6810 La Baroche,Verheiratet,518
9994,......6810 La Baroche,Verwitwet,75


Wir speichern den Output in einer separaten DF (als Kopie, damit wir nachher keinen Ärger kriegen).

In [24]:
df_g = df[df['Einheit'].str.contains(regex_g)].copy()

**Quiz:** Erstellen Sie zwei weitere Dataframes, in denen nur die Kantone und Bezirke sind.

In [25]:
df_k = df[df['Einheit'].str.contains(regex_k)].copy()

In [26]:
df_b = df[df['Einheit'].str.contains(regex_b)].copy()

Jetzt haben wir drei DataFrames mit den Daten zur Schweiz, den Kantonen und den Gemeinden. 

## 2. Daten extrahieren

Wir wenden uns den Gemeinden zu. Unser Ziel: Die Gemeindenummer in eine separate Spalte ziehen.

In [27]:
df_g.head(2)

,Einheit,Zivilstand,Anzahl
12,......0001 Aeugst am Albis,Ledig,852
13,......0001 Aeugst am Albis,Verheiratet,908


Hier kommt die Funktion `str.extract()` gelegen. Um sie zu benutzen, müssen wir zwei Dinge wissen:
1. die Regex, die zu dem gesamten Ausdruck passt.
2. Den Teil der Regex, den wir in Klammern `()` setzen und extrahieren wollen.

In [28]:
#df_g['Einheit'].str.extract(r"^\.{6}(\d{4}) .+$")#umständlich. Es geht auch einfacher:
df_g['Einheit'].str.extract(r"(\d{4})")

,0
12,0001
13,0001
14,0001
15,0001
16,0002
...,...
9991,6809
9992,6810
9993,6810
9994,6810


Wir können den Output dieser Funktion für die neue Spalte setzen.

In [29]:
df_g['Nummer'] = df_g['Einheit'].str.extract(r"(\d{4})")

In [30]:
df_g.head(10)

,Einheit,Zivilstand,Anzahl,Nummer
12,......0001 Aeugst am Albis,Ledig,852,0001
13,......0001 Aeugst am Albis,Verheiratet,908,0001
14,......0001 Aeugst am Albis,Verwitwet,74,0001
15,......0001 Aeugst am Albis,Geschieden,147,0001
16,......0002 Affoltern am Albis,Ledig,5000,0002
17,......0002 Affoltern am Albis,Verheiratet,5159,0002
18,......0002 Affoltern am Albis,Verwitwet,545,0002
19,......0002 Affoltern am Albis,Geschieden,1004,0002
20,......0003 Bonstetten,Ledig,2336,0003
21,......0003 Bonstetten,Verheiratet,2452,0003


**Quiz:** Erstellen Sie im Gemeinde-DF eine weitere Spalte ("Name"), die den Namen der Gemeinde enthält!

In [36]:
df_g['Gemeinde'] = df_g['Einheit'].str.extract(r"\.{6}\d{4}(.+)$")
df_g.head(5)

,Einheit,Zivilstand,Anzahl,Nummer,Gemeinde
12,......0001 Aeugst am Albis,Ledig,852,0001,Aeugst am Albis
13,......0001 Aeugst am Albis,Verheiratet,908,0001,Aeugst am Albis
14,......0001 Aeugst am Albis,Verwitwet,74,0001,Aeugst am Albis
15,......0001 Aeugst am Albis,Geschieden,147,0001,Aeugst am Albis
16,......0002 Affoltern am Albis,Ledig,5000,0002,Affoltern am Albis


In [39]:
df_g.pop('Einheit')

12         ......0001 Aeugst am Albis
13         ......0001 Aeugst am Albis
14         ......0001 Aeugst am Albis
15         ......0001 Aeugst am Albis
16      ......0002 Affoltern am Albis
                    ...              
9991           ......6809 Haute-Ajoie
9992            ......6810 La Baroche
9993            ......6810 La Baroche
9994            ......6810 La Baroche
9995            ......6810 La Baroche
Name: Einheit, Length: 9296, dtype: object

In [40]:
df_g

,Zivilstand,Anzahl,Nummer,Gemeinde
12,Ledig,852,0001,Aeugst am Albis
13,Verheiratet,908,0001,Aeugst am Albis
14,Verwitwet,74,0001,Aeugst am Albis
15,Geschieden,147,0001,Aeugst am Albis
16,Ledig,5000,0002,Affoltern am Albis
...,...,...,...,...
9991,Geschieden,69,6809,Haute-Ajoie
9992,Ledig,490,6810,La Baroche
9993,Verheiratet,518,6810,La Baroche
9994,Verwitwet,75,6810,La Baroche


## Übung

### Teil 1

**Hier haben Sie Gelegenheit noch etwas mit den Regex-Funktionen in Pandas zu experimentieren.**

Wir arbeiten wieder mit dem Haupt-Dataframe am Anfang: `df`.

**Aufgabe 1:** Erstellen Sie im DF eine zusätzliche Spalte mit dem Namen "Typ". Darin soll stehen, ob es sich bei der betreffenden Zeile um eine Gemeinde, einen Bezirk, einen Kanton oder ein Land handelt.

Tipp: Arbeiten Sie mit `df.loc[]`

In [ ]:
df['Typ'] = df.loc['Einheit']

**Aufgabe 2:** Bereinigen Sie die Bezeichnungen in der Spalte "Einheit". Wir wollen keine Nummern und keine Sonderzeichen mehr darin sehen, nur noch die Namen der Gemeinden, Bezirke und Kantone.

Tipp: Arbeiten Sie mit `str.replace()`

In [ ]:
# zum Testen, ob es bei den Kantonen funktioniert hat:
df[df['Typ'] == 'Kanton'].head(10)

In [ ]:
# zum Testen, ob es bei den Bezirken funktioniert hat:
df[df['Typ'] == 'Bezirk'].head(10)

In [ ]:
# zum Testen, ob es bei den Gemeinden funktioniert hat:
df[df['Typ'] == 'Gemeinde'].head(10)

### Teil 2

**Hier können Sie nochmals etwas Pivoting und Plotting trainieren**

In [ ]:
df.to_csv('dataprojects/BFS/Einheiten_kategorisiert.csv', index=False)

Zur Sicherheit laden wir die Daten nochmals neu - mit dem Stand der Übung bis hier.

In [ ]:
df = pd.read_csv('dataprojects/BFS/Einheiten_kategorisiert.csv')

**Aufgabe 1:** Finden Sie die drei Gemeinden mit der höchsten Quote an Verwitweten Personen in der Bevölkerung.

Tipp zum Vorgehen:
1. DF nach Gemeinden filtern
1. DF nach Zivilstand pivotieren
1. Total der Bevölkerung mit `sum()` ausrechnen (`axis=1`)
1. Verwitweten-Quote ausrechnen
1. DF sortieren

In [ ]:
# DF nach gemeinden filtern und pivotieren


In [ ]:
# Bevölkerungstotal ausrechnen pro Gemeinde


In [ ]:
# Verwitweten-Quote ausrechnen


In [ ]:
# Sortieren und Top 3


**Aufgabe 2:** Welcher Kanton hat die höchste Ledigen-Quote?

In [ ]:
# DF nach Kantonen filtern und pivotieren


In [ ]:
# Bevölkerungstotal ausrechnen pro Kanton


In [ ]:
# Ledigen-Quote ausrechnen


In [ ]:
# Sortieren und obersten anzeigen


**Aufgabe 3:** Zeichnen Sie einen Bar-Chart mit der Verheirateten-Quote aller Kantone

Tipp: Benutzen Sie das pivotierte DF, das Sie in Aufgabe 2 erstellt haben.

In [ ]:
# Verheirateten-Quote ausrechnen


In [ ]:
#Sortieren und plotten


**Aufgabe 4:** Zeichnen Sie ein Stacked-Bar-Chart mit den relativen Zivilstands-Anteilen in zwei Gemeinden:
- Corippo
- Meienried

Tipp zu Vorgehen:
1. DF nach Gemeinden filtern und pivotieren
1. Total der Einwohner für jede Gemeinde (in separater Series ablegen)
1. Pivotiertes DF mit `div()` durch das Total dividieren (`axis=1`)
1. mit `.loc[]` die beiden Gemeinden wählen
1. Plot mit `stacked=True`

In [ ]:
# DF nach gemeinden filtern und pivotieren


In [ ]:
# Total der Einwohner für jede Gemeinde


In [ ]:
# Neues DF mit relativen Werten


In [ ]:
# Nur Meienried und Corippo anzeigen


In [ ]:
# Plot
